<a href="https://colab.research.google.com/github/mevivekmevivek/Drug-Efficacy-Prediction/blob/main/copy_number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving modified-cnv_abs_copy_number_picnic_20191101.csv to modified-cnv_abs_copy_number_picnic_20191101.csv


In [ ]:
import io
copynumber = pd.read_csv(io.BytesIO(uploaded['modified-cnv_abs_copy_number_picnic_20191101.csv']))

In [ ]:
copynumber

,Unnamed: 0,M14,TE-12,TMK-1,STS-0421,PL4,PCI-4B,PCI-30,HSC-39,H3255,EMC-BAC-2,EMC-BAC-1,DiFi,CHSA0108,CHSA0011,HCE-4,MZ1-PC,PL18,201T,K5,HLE,BICR10,BICR31,SK-OV-3,SK-MEL-5,SK-MEL-28,SK-MEL-2,SF539,SF295,SF268,RXF393,RPMI-8226,PC-3,OVCAR-8,OVCAR-5,OVCAR-4,SN12C,SNB75,SR,T47D,...,SNU-387,SNU-398,UM-UC-3,UMC-11,T24,U-CH2,UACC-893,UACC-812,SU8686,U-87-MG,TCCSUP,U-2-OS,U-118-MG,SNU-449,SNU-475,SNU-C1,ST486,SNU-423,NK-92MI,TUR,IMR-5,KNS-81-FD,MY-M12,JEG-3,LU-99A,LNZTA3WT4,CHL-1,BE2-M17,C2BBe1,C3A,451Lu,MMAc-SF,BE-13,MC-IXC,Ramos-2G6-4C10,CGTH-W-1,H9,GR-ST,YMB-1-E,MM1S
0,A1BG,3.0,3.0,3.0,4.0,6.0,3.0,2.0,3.0,3.0,3.0,2.0,5.0,2.0,3.0,2.0,4.0,6.0,3.0,4.0,3.0,2.0,6.0,4.0,3.0,5.0,2.0,3.0,4.0,2.0,4.0,3.0,6.0,2.0,2.0,6.0,3.0,3.0,2.0,2.0,...,2.0,1.0,4.0,2.0,3.0,2.0,3.0,2.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,3.0,2.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,3.0,4.0,3.0,2.0,3.0,2.0,4.0,3.0,4.0,2.0,2.0,4.0,4.0,2.0,3.0,2.0
1,A1CF,3.0,3.0,3.0,4.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,5.0,3.0,2.0,2.0,4.0,5.0,3.0,4.0,3.0,2.0,3.0,4.0,2.0,3.0,2.0,3.0,2.0,2.0,3.0,2.0,10.0,3.0,2.0,4.0,3.0,2.0,2.0,4.0,...,2.0,4.0,3.0,3.0,3.0,2.0,4.0,3.0,2.0,3.0,4.0,4.0,2.0,4.0,4.0,2.0,2.0,7.0,3.0,2.0,2.0,1.0,2.0,3.0,3.0,3.0,3.0,2.0,6.0,3.0,4.0,2.0,3.0,1.0,2.0,3.0,4.0,2.0,3.0,2.0
2,A2M,3.0,3.0,2.0,4.0,9.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0,2.0,3.0,4.0,5.0,3.0,4.0,4.0,2.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,...,3.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,4.0,2.0,3.0,5.0,2.0,2.0,2.0,3.0,2.0,5.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,5.0,2.0,5.0,3.0,4.0,2.0,2.0,4.0,3.0,2.0,5.0,2.0
3,A2ML1,3.0,3.0,2.0,4.0,9.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0,2.0,3.0,4.0,5.0,3.0,4.0,4.0,2.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,...,3.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,4.0,2.0,3.0,5.0,2.0,2.0,2.0,3.0,2.0,5.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,5.0,2.0,5.0,3.0,4.0,2.0,2.0,4.0,3.0,2.0,5.0,2.0
4,A2ML1-AS1,3.0,3.0,2.0,4.0,9.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0,2.0,3.0,4.0,5.0,3.0,4.0,4.0,2.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,...,3.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,4.0,2.0,3.0,5.0,2.0,2.0,2.0,3.0,2.0,5.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,5.0,2.0,5.0,3.0,4.0,2.0,2.0,4.0,3.0,2.0,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24497,ZYG11AP1,3.0,3.0,3.0,4.0,4.0,4.0,3.0,3.0,2.0,2.0,2.0,4.0,4.0,2.0,3.0,3.0,6.0,4.0,4.0,2.0,2.0,3.0,4.0,4.0,4.0,2.0,3.0,3.0,2.0,3.0,2.0,4.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,...,4.0,2.0,3.0,2.0,4.0,1.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,3.0,3.0,4.0,2.0,7.0,3.0,2.0,2.0,2.0,2.0,5.0,4.0,3.0,3.0,2.0,4.0,2.0,2.0,4.0,2.0,2.0,2.0,4.0,3.0,2.0,3.0,2.0
24498,ZYG11B,4.0,3.0,3.0,4.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,2.0,2.0,5.0,3.0,7.0,3.0,4.0,3.0,2.0,3.0,4.0,3.0,5.0,2.0,4.0,3.0,3.0,3.0,4.0,5.0,3.0,2.0,3.0,3.0,3.0,2.0,2.0,...,3.0,3.0,4.0,3.0,4.0,1.0,3.0,3.0,3.0,2.0,3.0,3.0,2.0,2.0,3.0,3.0,2.0,5.0,4.0,3.0,4.0,2.0,2.0,3.0,2.0,4.0,4.0,1.0,3.0,2.0,5.0,3.0,4.0,2.0,2.0,4.0,3.0,2.0,2.0,2.0
24499,ZYX,4.0,3.0,3.0,5.0,3.0,2.0,3.0,4.0,5.0,2.0,3.0,4.0,4.0,2.0,2.0,3.0,12.0,4.0,4.0,2.0,1.0,3.0,2.0,6.0,5.0,3.0,3.0,4.0,3.0,4.0,4.0,6.0,3.0,2.0,5.0,3.0,4.0,3.0,2.0,...,3.0,3.0,4.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0,4.0,2.0,3.0,3.0,3.0,4.0,3.0,6.0,5.0,2.0,2.0,3.0,2.0,3.0,1.0,4.0,4.0,1.0,4.0,2.0,7.0,4.0,4.0,2.0,3.0,3.0,4.0,3.0,3.0,3.0
24500,ZZEF1,4.0,3.0,2.0,3.0,4.0,3.0,2.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,3.0,4.0,5.0,2.0,3.0,2.0,1.0,3.0,0.0,3.0,4.0,1.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,...,2.0,2.0,4.0,2.0,3.0,1.0,1.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,1.0,2.0,2.0,4.0,4.0,3.0,1.0,1.0,4.0,4.0,2.0,2.0,2.0


In [ ]:
copynumber.fillna(value = 0, inplace = True)

In [ ]:
percent_missing = copynumber.isnull().sum() * 100 / len(copynumber)
missing_value_df = pd.DataFrame({'column_name': copynumber.columns,
                                 'percent_missing': percent_missing})

In [ ]:
missing_value_df.sort_values('percent_missing', inplace=True)

In [ ]:
missing_value_df

,column_name,percent_missing
Unnamed: 0,Unnamed: 0,0.0
SW1463,SW1463,0.0
SW1116,SW1116,0.0
SW48,SW48,0.0
LoVo,LoVo,0.0
...,...,...
RCH-ACV,RCH-ACV,0.0
PL-21,PL-21,0.0
PF-382,PF-382,0.0
RH-18,RH-18,0.0


In [ ]:
copynumber_t = copynumber.T

In [ ]:
copynumber_t

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,24462,24463,24464,24465,24466,24467,24468,24469,24470,24471,24472,24473,24474,24475,24476,24477,24478,24479,24480,24481,24482,24483,24484,24485,24486,24487,24488,24489,24490,24491,24492,24493,24494,24495,24496,24497,24498,24499,24500,24501
Unnamed: 0,A1BG,A1CF,A2M,A2ML1,A2ML1-AS1,A2ML1-AS2,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,...,ZSCAN5B,ZSCAN5C,ZSCAN9,ZSCAN10,ZSCAN12,ZSCAN12P1,ZSCAN16,ZSCAN18,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZSWIM9,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3
M14,3,3,3,3,3,3,4,4,4,3,3,3,4,4,4,4,2,3,4,3,3,3,3,4,2,3,3,3,2,3,5,3,3,3,3,3,3,4,3,3,...,3,3,3,3,3,3,3,3,4,4,3,3,4,3,3,3,3,3,4,3,4,3,4,3,3,3,3,2,3,4,3,2,2,4,4,3,4,4,4,4
TE-12,3,3,3,3,3,3,2,3,3,3,3,2,3,3,3,3,2,3,4,3,4,3,4,5,4,4,2,3,2,4,4,3,4,2,3,3,2,3,4,4,...,3,3,2,2,2,2,2,3,2,4,3,2,4,2,5,2,2,2,5,3,5,3,3,2,3,3,3,2,3,4,3,2,2,3,3,3,3,3,3,3
TMK-1,3,3,2,2,2,2,2,2,3,3,3,2,3,3,3,3,2,3,3,2,4,2,3,3,3,3,3,3,2,4,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,2,3,3,3,3,3,3,4,3,3,3,2,3,3,2,3,2,2,3,2,4,3,3,2,2,1,3,3,3,3,2,3
STS-0421,4,4,4,4,4,4,4,4,5,4,4,5,5,5,4,4,4,4,4,4,3,3,3,3,5,4,3,3,4,3,4,3,3,4,4,3,4,4,3,3,...,4,4,3,4,3,3,3,4,4,5,4,3,5,3,2,3,3,4,3,3,3,4,4,4,3,5,4,3,3,4,5,2,2,5,4,4,4,5,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGTH-W-1,4,3,4,4,4,4,4,4,4,2,3,5,4,4,4,4,3,3,3,3,4,3,3,4,4,3,3,3,9,4,3,3,3,4,4,3,4,3,3,3,...,4,4,3,4,3,3,3,4,4,3,4,3,3,3,3,2,3,4,5,4,5,6,3,2,3,3,4,3,4,3,3,5,5,4,4,4,4,3,4,3
H9,4,4,3,3,3,3,3,3,3,3,3,2,3,3,3,3,3,4,3,3,2,3,4,4,2,3,5,4,2,2,4,4,4,3,3,5,2,3,4,4,...,4,4,3,2,3,3,3,4,3,4,4,3,4,3,2,4,3,2,4,2,4,2,3,2,4,4,8,4,2,3,3,2,2,4,3,3,3,4,4,3
GR-ST,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,3,2,2,2,2,...,2,2,2,2,2,2,2,2,2,3,2,2,3,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,1,1,2,2,2,2,3,2,2
YMB-1-E,3,3,5,5,5,5,2,2,3,3,3,3,3,3,3,3,3,3,4,3,2,3,4,4,4,5,3,3,3,2,3,3,4,4,3,3,4,2,5,5,...,3,3,3,4,3,3,3,3,2,3,3,3,3,3,4,4,3,4,4,3,4,3,3,3,2,4,2,1,2,4,3,3,3,3,2,3,2,3,2,2


In [ ]:
copynumber_t.columns=copynumber_t.iloc[0]

In [ ]:
copynumber_t= copynumber_t.iloc[1:,:]

In [ ]:
copynumber_t

Unnamed: 0,A1BG,A1CF,A2M,A2ML1,A2ML1-AS1,A2ML1-AS2,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,...,ZSCAN5B,ZSCAN5C,ZSCAN9,ZSCAN10,ZSCAN12,ZSCAN12P1,ZSCAN16,ZSCAN18,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZSWIM9,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3
M14,3,3,3,3,3,3,4,4,4,3,3,3,4,4,4,4,2,3,4,3,3,3,3,4,2,3,3,3,2,3,5,3,3,3,3,3,3,4,3,3,...,3,3,3,3,3,3,3,3,4,4,3,3,4,3,3,3,3,3,4,3,4,3,4,3,3,3,3,2,3,4,3,2,2,4,4,3,4,4,4,4
TE-12,3,3,3,3,3,3,2,3,3,3,3,2,3,3,3,3,2,3,4,3,4,3,4,5,4,4,2,3,2,4,4,3,4,2,3,3,2,3,4,4,...,3,3,2,2,2,2,2,3,2,4,3,2,4,2,5,2,2,2,5,3,5,3,3,2,3,3,3,2,3,4,3,2,2,3,3,3,3,3,3,3
TMK-1,3,3,2,2,2,2,2,2,3,3,3,2,3,3,3,3,2,3,3,2,4,2,3,3,3,3,3,3,2,4,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,2,3,3,3,3,3,3,4,3,3,3,2,3,3,2,3,2,2,3,2,4,3,3,2,2,1,3,3,3,3,2,3
STS-0421,4,4,4,4,4,4,4,4,5,4,4,5,5,5,4,4,4,4,4,4,3,3,3,3,5,4,3,3,4,3,4,3,3,4,4,3,4,4,3,3,...,4,4,3,4,3,3,3,4,4,5,4,3,5,3,2,3,3,4,3,3,3,4,4,4,3,5,4,3,3,4,5,2,2,5,4,4,4,5,3,4
PL4,6,3,9,9,9,9,3,5,6,4,4,4,6,6,3,3,4,4,4,4,6,6,5,8,4,4,5,5,4,6,3,5,5,4,4,4,4,5,5,5,...,6,6,5,4,5,5,5,6,3,3,6,5,3,5,4,2,5,4,8,6,8,4,3,6,4,4,6,3,5,4,4,3,3,4,3,4,3,3,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGTH-W-1,4,3,4,4,4,4,4,4,4,2,3,5,4,4,4,4,3,3,3,3,4,3,3,4,4,3,3,3,9,4,3,3,3,4,4,3,4,3,3,3,...,4,4,3,4,3,3,3,4,4,3,4,3,3,3,3,2,3,4,5,4,5,6,3,2,3,3,4,3,4,3,3,5,5,4,4,4,4,3,4,3
H9,4,4,3,3,3,3,3,3,3,3,3,2,3,3,3,3,3,4,3,3,2,3,4,4,2,3,5,4,2,2,4,4,4,3,3,5,2,3,4,4,...,4,4,3,2,3,3,3,4,3,4,4,3,4,3,2,4,3,2,4,2,4,2,3,2,4,4,8,4,2,3,3,2,2,4,3,3,3,4,4,3
GR-ST,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,3,2,2,2,2,...,2,2,2,2,2,2,2,2,2,3,2,2,3,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,1,1,2,2,2,2,3,2,2
YMB-1-E,3,3,5,5,5,5,2,2,3,3,3,3,3,3,3,3,3,3,4,3,2,3,4,4,4,5,3,3,3,2,3,3,4,4,3,3,4,2,5,5,...,3,3,3,4,3,3,3,3,2,3,3,3,3,3,4,4,3,4,4,3,4,3,3,3,2,4,2,1,2,4,3,3,3,3,2,3,2,3,2,2


In [ ]:
y = copynumber_t.index

In [ ]:
y

Index(['M14', 'TE-12', 'TMK-1', 'STS-0421', 'PL4', 'PCI-4B', 'PCI-30',
       'HSC-39', 'H3255', 'EMC-BAC-2',
       ...
       '451Lu', 'MMAc-SF', 'BE-13', 'MC-IXC', 'Ramos-2G6-4C10', 'CGTH-W-1',
       'H9', 'GR-ST', 'YMB-1-E', 'MM1S'],
      dtype='object', length=986)

In [ ]:
copynumber_t.insert(0, 'CELL_LINE_NAME', y)

In [ ]:
copynumber_t

Unnamed: 0,CELL_LINE_NAME,A1BG,A1CF,A2M,A2ML1,A2ML1-AS1,A2ML1-AS2,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZSCAN5B,ZSCAN5C,ZSCAN9,ZSCAN10,ZSCAN12,ZSCAN12P1,ZSCAN16,ZSCAN18,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZSWIM9,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3
M14,M14,3,3,3,3,3,3,4,4,4,3,3,3,4,4,4,4,2,3,4,3,3,3,3,4,2,3,3,3,2,3,5,3,3,3,3,3,3,4,3,...,3,3,3,3,3,3,3,3,4,4,3,3,4,3,3,3,3,3,4,3,4,3,4,3,3,3,3,2,3,4,3,2,2,4,4,3,4,4,4,4
TE-12,TE-12,3,3,3,3,3,3,2,3,3,3,3,2,3,3,3,3,2,3,4,3,4,3,4,5,4,4,2,3,2,4,4,3,4,2,3,3,2,3,4,...,3,3,2,2,2,2,2,3,2,4,3,2,4,2,5,2,2,2,5,3,5,3,3,2,3,3,3,2,3,4,3,2,2,3,3,3,3,3,3,3
TMK-1,TMK-1,3,3,2,2,2,2,2,2,3,3,3,2,3,3,3,3,2,3,3,2,4,2,3,3,3,3,3,3,2,4,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,2,3,3,3,3,3,3,4,3,3,3,2,3,3,2,3,2,2,3,2,4,3,3,2,2,1,3,3,3,3,2,3
STS-0421,STS-0421,4,4,4,4,4,4,4,4,5,4,4,5,5,5,4,4,4,4,4,4,3,3,3,3,5,4,3,3,4,3,4,3,3,4,4,3,4,4,3,...,4,4,3,4,3,3,3,4,4,5,4,3,5,3,2,3,3,4,3,3,3,4,4,4,3,5,4,3,3,4,5,2,2,5,4,4,4,5,3,4
PL4,PL4,6,3,9,9,9,9,3,5,6,4,4,4,6,6,3,3,4,4,4,4,6,6,5,8,4,4,5,5,4,6,3,5,5,4,4,4,4,5,5,...,6,6,5,4,5,5,5,6,3,3,6,5,3,5,4,2,5,4,8,6,8,4,3,6,4,4,6,3,5,4,4,3,3,4,3,4,3,3,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGTH-W-1,CGTH-W-1,4,3,4,4,4,4,4,4,4,2,3,5,4,4,4,4,3,3,3,3,4,3,3,4,4,3,3,3,9,4,3,3,3,4,4,3,4,3,3,...,4,4,3,4,3,3,3,4,4,3,4,3,3,3,3,2,3,4,5,4,5,6,3,2,3,3,4,3,4,3,3,5,5,4,4,4,4,3,4,3
H9,H9,4,4,3,3,3,3,3,3,3,3,3,2,3,3,3,3,3,4,3,3,2,3,4,4,2,3,5,4,2,2,4,4,4,3,3,5,2,3,4,...,4,4,3,2,3,3,3,4,3,4,4,3,4,3,2,4,3,2,4,2,4,2,3,2,4,4,8,4,2,3,3,2,2,4,3,3,3,4,4,3
GR-ST,GR-ST,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,3,2,2,2,...,2,2,2,2,2,2,2,2,2,3,2,2,3,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,1,1,2,2,2,2,3,2,2
YMB-1-E,YMB-1-E,3,3,5,5,5,5,2,2,3,3,3,3,3,3,3,3,3,3,4,3,2,3,4,4,4,5,3,3,3,2,3,3,4,4,3,3,4,2,5,...,3,3,3,4,3,3,3,3,2,3,3,3,3,3,4,4,3,4,4,3,4,3,3,3,2,4,2,1,2,4,3,3,3,3,2,3,2,3,2,2


In [ ]:
copynumber_t.columns

Index(['CELL_LINE_NAME', 'A1BG', 'A1CF', 'A2M', 'A2ML1', 'A2ML1-AS1',
       'A2ML1-AS2', 'A3GALT2', 'A4GALT', 'A4GNT',
       ...
       'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11AP1', 'ZYG11B', 'ZYX',
       'ZZEF1', 'ZZZ3'],
      dtype='object', name='Unnamed: 0', length=24503)

In [ ]:
copynumber_t.info()

<class 'pandas.core.frame.DataFrame'>
Index: 986 entries, M14 to MM1S
Columns: 24503 entries, CELL_LINE_NAME to ZZZ3
dtypes: object(24503)
memory usage: 184.3+ MB


In [ ]:
y = copynumber_t['CELL_LINE_NAME']
copynumber_t.drop(columns =['CELL_LINE_NAME'], inplace =True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
import numpy as np
for p in copynumber_t.columns:
    
    copynumber_t[p] = copynumber_t[p].astype(np.uint8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
copynumber_t.info()

<class 'pandas.core.frame.DataFrame'>
Index: 986 entries, M14 to MM1S
Columns: 24502 entries, A1BG to ZZZ3
dtypes: uint8(24502)
memory usage: 23.0+ MB


In [ ]:
copynumber_t.insert(0,"CELL_LINE_NAME", y) 

In [ ]:
from google.colab import files
copynumber_t.to_csv('copynumber.csv') 
files.download('copynumber.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
copynumber_t

Unnamed: 0,CELL_LINE_NAME,A1BG,A1CF,A2M,A2ML1,A2ML1-AS1,A2ML1-AS2,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZSCAN5B,ZSCAN5C,ZSCAN9,ZSCAN10,ZSCAN12,ZSCAN12P1,ZSCAN16,ZSCAN18,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZSWIM9,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3
M14,M14,3,3,3,3,3,3,4,4,4,3,3,3,4,4,4,4,2,3,4,3,3,3,3,4,2,3,3,3,2,3,5,3,3,3,3,3,3,4,3,...,3,3,3,3,3,3,3,3,4,4,3,3,4,3,3,3,3,3,4,3,4,3,4,3,3,3,3,2,3,4,3,2,2,4,4,3,4,4,4,4
TE-12,TE-12,3,3,3,3,3,3,2,3,3,3,3,2,3,3,3,3,2,3,4,3,4,3,4,5,4,4,2,3,2,4,4,3,4,2,3,3,2,3,4,...,3,3,2,2,2,2,2,3,2,4,3,2,4,2,5,2,2,2,5,3,5,3,3,2,3,3,3,2,3,4,3,2,2,3,3,3,3,3,3,3
TMK-1,TMK-1,3,3,2,2,2,2,2,2,3,3,3,2,3,3,3,3,2,3,3,2,4,2,3,3,3,3,3,3,2,4,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,2,3,3,3,3,3,3,4,3,3,3,2,3,3,2,3,2,2,3,2,4,3,3,2,2,1,3,3,3,3,2,3
STS-0421,STS-0421,4,4,4,4,4,4,4,4,5,4,4,5,5,5,4,4,4,4,4,4,3,3,3,3,5,4,3,3,4,3,4,3,3,4,4,3,4,4,3,...,4,4,3,4,3,3,3,4,4,5,4,3,5,3,2,3,3,4,3,3,3,4,4,4,3,5,4,3,3,4,5,2,2,5,4,4,4,5,3,4
PL4,PL4,6,3,9,9,9,9,3,5,6,4,4,4,6,6,3,3,4,4,4,4,6,6,5,8,4,4,5,5,4,6,3,5,5,4,4,4,4,5,5,...,6,6,5,4,5,5,5,6,3,3,6,5,3,5,4,2,5,4,8,6,8,4,3,6,4,4,6,3,5,4,4,3,3,4,3,4,3,3,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGTH-W-1,CGTH-W-1,4,3,4,4,4,4,4,4,4,2,3,5,4,4,4,4,3,3,3,3,4,3,3,4,4,3,3,3,9,4,3,3,3,4,4,3,4,3,3,...,4,4,3,4,3,3,3,4,4,3,4,3,3,3,3,2,3,4,5,4,5,6,3,2,3,3,4,3,4,3,3,5,5,4,4,4,4,3,4,3
H9,H9,4,4,3,3,3,3,3,3,3,3,3,2,3,3,3,3,3,4,3,3,2,3,4,4,2,3,5,4,2,2,4,4,4,3,3,5,2,3,4,...,4,4,3,2,3,3,3,4,3,4,4,3,4,3,2,4,3,2,4,2,4,2,3,2,4,4,8,4,2,3,3,2,2,4,3,3,3,4,4,3
GR-ST,GR-ST,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,3,2,2,2,...,2,2,2,2,2,2,2,2,2,3,2,2,3,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,1,1,2,2,2,2,3,2,2
YMB-1-E,YMB-1-E,3,3,5,5,5,5,2,2,3,3,3,3,3,3,3,3,3,3,4,3,2,3,4,4,4,5,3,3,3,2,3,3,4,4,3,3,4,2,5,...,3,3,3,4,3,3,3,3,2,3,3,3,3,3,4,4,3,4,4,3,4,3,3,3,2,4,2,1,2,4,3,3,3,3,2,3,2,3,2,2


In [ ]:
import pickle

In [ ]:
pickle_out = open("copynumber.pickle", "wb")


In [ ]:
pickle.dump(copynumber_t,pickle_out)
pickle_out.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print (~copynumber_t.columns.duplicated())
copynumber_t = copynumber_t.loc[:, ~copynumber_t.columns.duplicated()]


[ True  True  True ...  True  True  True]


In [ ]:
copynumber_t.info()

<class 'pandas.core.frame.DataFrame'>
Index: 986 entries, M14 to MM1S
Columns: 24503 entries, CELL_LINE_NAME to ZZZ3
dtypes: object(1), uint8(24502)
memory usage: 23.1+ MB


In [ ]:
copynumber_same_value =[]
copynumber_new_features=[]
for x in copynumber_t.columns:
    if copynumber_t[x].nunique() ==1:
        copynumber_same_value.append(x)
    else :
        copynumber_new_features.append(x)

In [ ]:
print(len(copynumber_same_value))
print(len(copynumber_new_features))

0
24503


In [ ]:
copynumber_t.isnull().values.any()

False

In [ ]:
copynumber_y = copynumber_t['CELL_LINE_NAME']
copynumber_t.drop(columns =['CELL_LINE_NAME'], inplace = True)
copynumber_y

M14              M14
TE-12          TE-12
TMK-1          TMK-1
STS-0421    STS-0421
PL4              PL4
              ...   
CGTH-W-1    CGTH-W-1
H9                H9
GR-ST          GR-ST
YMB-1-E      YMB-1-E
MM1S            MM1S
Name: CELL_LINE_NAME, Length: 986, dtype: object

In [ ]:
from sklearn.preprocessing import minmax_scale
copynumber_scaled = minmax_scale(copynumber_t, axis = 0)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(copynumber_scaled, copynumber_y, train_size = 0.9, random_state = seed(2017))

In [ ]:
mcol = X_train.shape[1]
encoding_dim1 = 100

In [ ]:
from tensorflow.keras.layers import LeakyReLU
input_dim1 = Input(shape = (mcol, ))

# Encoder Layers
#encoded1 = Dense(20000)(input_dim1)
#encoded1= LeakyReLU(alpha=0.3)(encoded1)
#encoded2 = Dense(15000, activation = 'relu')(input_dim1)
#encoded3 = Dense(10000, activation = 'relu')(encoded2)
#encoded4 = Dense(5000)(encoded1)
#encoded4= LeakyReLU(alpha=0.3)(encoded4)
#encoded5 = Dense(2000, activation = 'relu')(encoded2)
#encoded6 = Dense(1750, activation = 'relu')(encoded5)
#encoded7 = Dense(1500, activation = 'relu')(encoded6)
#encoded8 = Dense(1250, activation = 'relu')(encoded7)
#encoded9 = Dense(1000)(encoded1)
#encoded9= LeakyReLU(alpha=0.3)(encoded9)
#encoded10 = Dense(750, activation = 'relu')(encoded9)
#encoded11 = Dense(500, activation = 'relu')(encoded10)
#encoded12 = Dense(250, activation = 'relu')(encoded9)
encoded13 = Dense(encoding_dim1)(input_dim1)
encoded13= LeakyReLU(alpha=0.3)(encoded13)

encoder1 = Model(inputs = input_dim1, outputs = encoded13)

# Decoder Layers
#decoded1 = Dense(mcol, activation = 'relu')(encoded13)
#decoded2 = Dense(500, activation = 'relu')(decoded1)
#decoded3 = Dense(750, activation = 'relu')(decoded2)
#decoded4 = Dense(1000, activation = 'relu')(encoded13)
#decoded5 = Dense(1250, activation = 'relu')(decoded4)
#decoded6 = Dense(1500, activation = 'relu')(decoded5)
#decoded7 = Dense(1750, activation = 'relu')(decoded6)
#decoded8 = Dense(2000, activation = 'relu')(encoded13)
#decoded9 = Dense(5000, activation = 'relu')(decoded4)
#decoded10 = Dense(10000, activation = 'relu')(decoded9)
#decoded11 = Dense(15000, activation = 'relu')(decoded1)
#decoded12 = Dense(20000, activation = 'relu')(encoded13)
decoded13 = Dense(mcol, activation = 'relu')(encoded13)

# Combine Encoder and Deocder layers
autoencoder1 = Model(inputs = input_dim1, outputs = decoded13)

# Compile the Model
autoencoder1.compile(optimizer = 'adam', loss = 'mae')

In [ ]:
autoencoder1.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24502)]           0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2450300   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 24502)             2474702   
Total params: 4,925,002
Trainable params: 4,925,002
Non-trainable params: 0
_________________________________________________________________


In [ ]:
autoencoder1.fit(X_train, X_train, epochs = 300, batch_size = 32, shuffle = False, validation_data = (X_test, X_test))

Epoch 1/300
28/28 [==============================] - 0s 8ms/step - loss: 0.1527 - val_loss: 0.1154
Epoch 2/300
28/28 [==============================] - 0s 5ms/step - loss: 0.1121 - val_loss: 0.1129
Epoch 3/300
28/28 [==============================] - 0s 5ms/step - loss: 0.1057 - val_loss: 0.1036
Epoch 4/300
28/28 [==============================] - 0s 5ms/step - loss: 0.1001 - val_loss: 0.0976
Epoch 5/300
28/28 [==============================] - 0s 5ms/step - loss: 0.0988 - val_loss: 0.0956
Epoch 6/300
28/28 [==============================] - 0s 5ms/step - loss: 0.0945 - val_loss: 0.1011
Epoch 7/300
28/28 [==============================] - 0s 5ms/step - loss: 0.0924 - val_loss: 0.0866
Epoch 8/300
28/28 [==============================] - 0s 5ms/step - loss: 0.0865 - val_loss: 0.0885
Epoch 9/300
28/28 [==============================] - 0s 5ms/step - loss: 0.0823 - val_loss: 0.0927
Epoch 10/300
28/28 [==============================] - 0s 5ms/step - loss: 0.0812 - val_loss: 0.0863
Epoch 11/

In [ ]:
encoded_input1 = Input(shape = (encoding_dim1, ))

In [ ]:
encoded_train1 = pd.DataFrame(encoder1.predict(copynumber_scaled))
encoded_train1 = encoded_train1.add_prefix('copynumber_feature_')

In [ ]:
encoded_train_columns = encoded_train1.columns
for l in encoded_train_columns:
  if (encoded_train1[l] == 0).all():
    print("TRUE", l)

In [ ]:
encoded_train1

,copynumber_feature_0,copynumber_feature_1,copynumber_feature_2,copynumber_feature_3,copynumber_feature_4,copynumber_feature_5,copynumber_feature_6,copynumber_feature_7,copynumber_feature_8,copynumber_feature_9,copynumber_feature_10,copynumber_feature_11,copynumber_feature_12,copynumber_feature_13,copynumber_feature_14,copynumber_feature_15,copynumber_feature_16,copynumber_feature_17,copynumber_feature_18,copynumber_feature_19,copynumber_feature_20,copynumber_feature_21,copynumber_feature_22,copynumber_feature_23,copynumber_feature_24,copynumber_feature_25,copynumber_feature_26,copynumber_feature_27,copynumber_feature_28,copynumber_feature_29,copynumber_feature_30,copynumber_feature_31,copynumber_feature_32,copynumber_feature_33,copynumber_feature_34,copynumber_feature_35,copynumber_feature_36,copynumber_feature_37,copynumber_feature_38,copynumber_feature_39,...,copynumber_feature_60,copynumber_feature_61,copynumber_feature_62,copynumber_feature_63,copynumber_feature_64,copynumber_feature_65,copynumber_feature_66,copynumber_feature_67,copynumber_feature_68,copynumber_feature_69,copynumber_feature_70,copynumber_feature_71,copynumber_feature_72,copynumber_feature_73,copynumber_feature_74,copynumber_feature_75,copynumber_feature_76,copynumber_feature_77,copynumber_feature_78,copynumber_feature_79,copynumber_feature_80,copynumber_feature_81,copynumber_feature_82,copynumber_feature_83,copynumber_feature_84,copynumber_feature_85,copynumber_feature_86,copynumber_feature_87,copynumber_feature_88,copynumber_feature_89,copynumber_feature_90,copynumber_feature_91,copynumber_feature_92,copynumber_feature_93,copynumber_feature_94,copynumber_feature_95,copynumber_feature_96,copynumber_feature_97,copynumber_feature_98,copynumber_feature_99
0,-0.109201,-0.160136,-2.531141,0.021019,-0.378449,-0.291698,-0.690806,-1.068353,-1.038360,-0.173727,-1.076319,-0.045424,-0.906190,-0.185409,-0.418334,2.213546,-0.834868,-0.727916,-0.397759,-1.547042,-1.257846,1.498309,-0.486115,-1.565255,-0.531676,-2.419325,-0.763678,-0.928130,-0.776224,-0.738003,-0.822189,-1.254953,-1.043374,-0.601560,-0.913309,-0.377365,-0.220893,-0.843575,-0.312568,-1.294610,...,-0.106526,-1.561188,-1.569359,-0.152399,-0.094894,-0.726564,-0.200979,-1.470503,-0.316897,-0.103225,-0.298317,0.766120,0.394474,-0.497653,-0.786021,-1.384035,-0.377384,-0.630236,-0.832075,-0.214613,0.428468,1.346375,-1.213028,-1.425219,-0.671785,-0.885237,-0.637114,-0.231003,-0.835255,-0.396672,-0.872723,-1.486059,-1.016892,-1.526652,1.596862,-0.744194,-0.521416,-1.181389,-1.295292,-0.760382
1,-0.325570,-0.785261,-3.176779,-0.795387,-0.259566,-0.321688,-0.215838,-1.145755,-0.361033,-0.009557,-1.479106,0.255681,-0.897880,-0.403064,-0.181555,1.436580,-0.159714,-0.738293,-0.387282,-0.970099,-0.827163,-0.753069,1.764941,-1.595275,-0.398178,-1.872913,0.681076,-0.638815,-0.430512,1.674277,-0.527286,-0.961032,-1.422899,0.256682,-1.575647,-1.474825,0.801652,-0.888368,-2.469102,-1.140743,...,-0.636845,-0.484389,-0.658274,-0.264124,-0.892126,-0.886500,-1.244537,-0.243777,-0.387718,-1.440380,-0.401305,-0.632281,1.306335,-0.875206,-1.194235,-1.776081,-0.563451,-0.068355,-0.471638,3.186594,-0.731198,-0.427989,-0.715988,-1.001317,-0.427283,-0.164892,-0.938388,-0.493095,-1.000592,0.447484,-0.123566,-4.175654,-0.891017,-2.647295,-0.150808,-0.408331,-1.310105,-3.441621,-0.687869,-0.007424
2,-0.156567,-0.727474,-0.539222,-0.709865,-0.113751,-0.351196,-0.165670,-1.425358,-0.616899,-0.308831,-0.176213,0.068980,-1.436771,-0.227062,-0.431906,-0.069762,-0.155983,-0.938756,-0.848487,-1.268031,-0.756890,2.248810,-0.170544,-2.294326,-0.121383,-0.193952,-0.690122,-1.299559,-0.706316,0.457038,-0.209160,-0.802947,-0.767524,-0.285864,-0.703230,-1.172867,0.523440,-1.122247,-0.898149,-1.489581,...,-0.082646,-1.254114,-0.173183,-1.124363,-0.323065,-0.692566,0.435021,-0.422154,-0.258285,0.149669,-0.796491,0.332246,-0.036642,-0.893593,-1.170328,-1.147068,-0.593673,-0.036056,-0.589180,-0.154927,-0.034697,-0.135707,-0.242190,-0.453671,-0.618154,-0

In [ ]:

encoded_train1.insert(0, 'CELL_LINE_NAME', copynumber_y)
encoded_train1

,CELL_LINE_NAME,copynumber_feature_0,copynumber_feature_1,copynumber_feature_2,copynumber_feature_3,copynumber_feature_4,copynumber_feature_5,copynumber_feature_6,copynumber_feature_7,copynumber_feature_8,copynumber_feature_9,copynumber_feature_10,copynumber_feature_11,copynumber_feature_12,copynumber_feature_13,copynumber_feature_14,copynumber_feature_15,copynumber_feature_16,copynumber_feature_17,copynumber_feature_18,copynumber_feature_19,copynumber_feature_20,copynumber_feature_21,copynumber_feature_22,copynumber_feature_23,copynumber_feature_24,copynumber_feature_25,copynumber_feature_26,copynumber_feature_27,copynumber_feature_28,copynumber_feature_29,copynumber_feature_30,copynumber_feature_31,copynumber_feature_32,copynumber_feature_33,copynumber_feature_34,copynumber_feature_35,copynumber_feature_36,copynumber_feature_37,copynumber_feature_38,...,copynumber_feature_60,copynumber_feature_61,copynumber_feature_62,copynumber_feature_63,copynumber_feature_64,copynumber_feature_65,copynumber_feature_66,copynumber_feature_67,copynumber_feature_68,copynumber_feature_69,copynumber_feature_70,copynumber_feature_71,copynumber_feature_72,copynumber_feature_73,copynumber_feature_74,copynumber_feature_75,copynumber_feature_76,copynumber_feature_77,copynumber_feature_78,copynumber_feature_79,copynumber_feature_80,copynumber_feature_81,copynumber_feature_82,copynumber_feature_83,copynumber_feature_84,copynumber_feature_85,copynumber_feature_86,copynumber_feature_87,copynumber_feature_88,copynumber_feature_89,copynumber_feature_90,copynumber_feature_91,copynumber_feature_92,copynumber_feature_93,copynumber_feature_94,copynumber_feature_95,copynumber_feature_96,copynumber_feature_97,copynumber_feature_98,copynumber_feature_99
0,NaN,-0.109201,-0.160136,-2.531141,0.021019,-0.378449,-0.291698,-0.690806,-1.068353,-1.038360,-0.173727,-1.076319,-0.045424,-0.906190,-0.185409,-0.418334,2.213546,-0.834868,-0.727916,-0.397759,-1.547042,-1.257846,1.498309,-0.486115,-1.565255,-0.531676,-2.419325,-0.763678,-0.928130,-0.776224,-0.738003,-0.822189,-1.254953,-1.043374,-0.601560,-0.913309,-0.377365,-0.220893,-0.843575,-0.312568,...,-0.106526,-1.561188,-1.569359,-0.152399,-0.094894,-0.726564,-0.200979,-1.470503,-0.316897,-0.103225,-0.298317,0.766120,0.394474,-0.497653,-0.786021,-1.384035,-0.377384,-0.630236,-0.832075,-0.214613,0.428468,1.346375,-1.213028,-1.425219,-0.671785,-0.885237,-0.637114,-0.231003,-0.835255,-0.396672,-0.872723,-1.486059,-1.016892,-1.526652,1.596862,-0.744194,-0.521416,-1.181389,-1.295292,-0.760382
1,NaN,-0.325570,-0.785261,-3.176779,-0.795387,-0.259566,-0.321688,-0.215838,-1.145755,-0.361033,-0.009557,-1.479106,0.255681,-0.897880,-0.403064,-0.181555,1.436580,-0.159714,-0.738293,-0.387282,-0.970099,-0.827163,-0.753069,1.764941,-1.595275,-0.398178,-1.872913,0.681076,-0.638815,-0.430512,1.674277,-0.527286,-0.961032,-1.422899,0.256682,-1.575647,-1.474825,0.801652,-0.888368,-2.469102,...,-0.636845,-0.484389,-0.658274,-0.264124,-0.892126,-0.886500,-1.244537,-0.243777,-0.387718,-1.440380,-0.401305,-0.632281,1.306335,-0.875206,-1.194235,-1.776081,-0.563451,-0.068355,-0.471638,3.186594,-0.731198,-0.427989,-0.715988,-1.001317,-0.427283,-0.164892,-0.938388,-0.493095,-1.000592,0.447484,-0.123566,-4.175654,-0.891017,-2.647295,-0.150808,-0.408331,-1.310105,-3.441621,-0.687869,-0.007424
2,NaN,-0.156567,-0.727474,-0.539222,-0.709865,-0.113751,-0.351196,-0.165670,-1.425358,-0.616899,-0.308831,-0.176213,0.068980,-1.436771,-0.227062,-0.431906,-0.069762,-0.155983,-0.938756,-0.848487,-1.268031,-0.756890,2.248810,-0.170544,-2.294326,-0.121383,-0.193952,-0.690122,-1.299559,-0.706316,0.457038,-0.209160,-0.802947,-0.767524,-0.285864,-0.703230,-1.172867,0.523440,-1.122247,-0.898149,...,-0.082646,-1.254114,-0.173183,-1.124363,-0.323065,-0.692566,0.435021,-0.422154,-0.258285,0.149669,-0.796491,0.332246,-0.036642,-0.893593,-1.170328,-1.147068,-0.593673,-0.036056,-0.589180,-0.154927,-0.034697,-0.135707,-0.242190,-0.453671,-0.618154,-0.843877,-1.799632,0.27414

In [ ]:
encoded_train1['Cell_line'] = copynumber_y.values

In [ ]:
encoded_train1

,CELL_LINE_NAME,copynumber_feature_0,copynumber_feature_1,copynumber_feature_2,copynumber_feature_3,copynumber_feature_4,copynumber_feature_5,copynumber_feature_6,copynumber_feature_7,copynumber_feature_8,copynumber_feature_9,copynumber_feature_10,copynumber_feature_11,copynumber_feature_12,copynumber_feature_13,copynumber_feature_14,copynumber_feature_15,copynumber_feature_16,copynumber_feature_17,copynumber_feature_18,copynumber_feature_19,copynumber_feature_20,copynumber_feature_21,copynumber_feature_22,copynumber_feature_23,copynumber_feature_24,copynumber_feature_25,copynumber_feature_26,copynumber_feature_27,copynumber_feature_28,copynumber_feature_29,copynumber_feature_30,copynumber_feature_31,copynumber_feature_32,copynumber_feature_33,copynumber_feature_34,copynumber_feature_35,copynumber_feature_36,copynumber_feature_37,copynumber_feature_38,...,copynumber_feature_61,copynumber_feature_62,copynumber_feature_63,copynumber_feature_64,copynumber_feature_65,copynumber_feature_66,copynumber_feature_67,copynumber_feature_68,copynumber_feature_69,copynumber_feature_70,copynumber_feature_71,copynumber_feature_72,copynumber_feature_73,copynumber_feature_74,copynumber_feature_75,copynumber_feature_76,copynumber_feature_77,copynumber_feature_78,copynumber_feature_79,copynumber_feature_80,copynumber_feature_81,copynumber_feature_82,copynumber_feature_83,copynumber_feature_84,copynumber_feature_85,copynumber_feature_86,copynumber_feature_87,copynumber_feature_88,copynumber_feature_89,copynumber_feature_90,copynumber_feature_91,copynumber_feature_92,copynumber_feature_93,copynumber_feature_94,copynumber_feature_95,copynumber_feature_96,copynumber_feature_97,copynumber_feature_98,copynumber_feature_99,Cell_line
0,NaN,-0.109201,-0.160136,-2.531141,0.021019,-0.378449,-0.291698,-0.690806,-1.068353,-1.038360,-0.173727,-1.076319,-0.045424,-0.906190,-0.185409,-0.418334,2.213546,-0.834868,-0.727916,-0.397759,-1.547042,-1.257846,1.498309,-0.486115,-1.565255,-0.531676,-2.419325,-0.763678,-0.928130,-0.776224,-0.738003,-0.822189,-1.254953,-1.043374,-0.601560,-0.913309,-0.377365,-0.220893,-0.843575,-0.312568,...,-1.561188,-1.569359,-0.152399,-0.094894,-0.726564,-0.200979,-1.470503,-0.316897,-0.103225,-0.298317,0.766120,0.394474,-0.497653,-0.786021,-1.384035,-0.377384,-0.630236,-0.832075,-0.214613,0.428468,1.346375,-1.213028,-1.425219,-0.671785,-0.885237,-0.637114,-0.231003,-0.835255,-0.396672,-0.872723,-1.486059,-1.016892,-1.526652,1.596862,-0.744194,-0.521416,-1.181389,-1.295292,-0.760382,M14
1,NaN,-0.325570,-0.785261,-3.176779,-0.795387,-0.259566,-0.321688,-0.215838,-1.145755,-0.361033,-0.009557,-1.479106,0.255681,-0.897880,-0.403064,-0.181555,1.436580,-0.159714,-0.738293,-0.387282,-0.970099,-0.827163,-0.753069,1.764941,-1.595275,-0.398178,-1.872913,0.681076,-0.638815,-0.430512,1.674277,-0.527286,-0.961032,-1.422899,0.256682,-1.575647,-1.474825,0.801652,-0.888368,-2.469102,...,-0.484389,-0.658274,-0.264124,-0.892126,-0.886500,-1.244537,-0.243777,-0.387718,-1.440380,-0.401305,-0.632281,1.306335,-0.875206,-1.194235,-1.776081,-0.563451,-0.068355,-0.471638,3.186594,-0.731198,-0.427989,-0.715988,-1.001317,-0.427283,-0.164892,-0.938388,-0.493095,-1.000592,0.447484,-0.123566,-4.175654,-0.891017,-2.647295,-0.150808,-0.408331,-1.310105,-3.441621,-0.687869,-0.007424,TE-12
2,NaN,-0.156567,-0.727474,-0.539222,-0.709865,-0.113751,-0.351196,-0.165670,-1.425358,-0.616899,-0.308831,-0.176213,0.068980,-1.436771,-0.227062,-0.431906,-0.069762,-0.155983,-0.938756,-0.848487,-1.268031,-0.756890,2.248810,-0.170544,-2.294326,-0.121383,-0.193952,-0.690122,-1.299559,-0.706316,0.457038,-0.209160,-0.802947,-0.767524,-0.285864,-0.703230,-1.172867,0.523440,-1.122247,-0.898149,...,-1.254114,-0.173183,-1.124363,-0.323065,-0.692566,0.435021,-0.422154,-0.258285,0.149669,-0.796491,0.332246,-0.036642,-0.893593,-1.170328,-1.147068,-0.593673,-0.036056,-0.589180,-0.154927,-0.034697,-0.135707,-0.242190,-0.453671,-0.618154,-0.843877,-1.799632,0.274148,-0.773049,-0.854677,-0.564587,

In [ ]:
encoded_train1.drop(columns = ['CELL_LINE_NAME'], inplace = True)

In [ ]:
encoded_train1

,copynumber_feature_0,copynumber_feature_1,copynumber_feature_2,copynumber_feature_3,copynumber_feature_4,copynumber_feature_5,copynumber_feature_6,copynumber_feature_7,copynumber_feature_8,copynumber_feature_9,copynumber_feature_10,copynumber_feature_11,copynumber_feature_12,copynumber_feature_13,copynumber_feature_14,copynumber_feature_15,copynumber_feature_16,copynumber_feature_17,copynumber_feature_18,copynumber_feature_19,copynumber_feature_20,copynumber_feature_21,copynumber_feature_22,copynumber_feature_23,copynumber_feature_24,copynumber_feature_25,copynumber_feature_26,copynumber_feature_27,copynumber_feature_28,copynumber_feature_29,copynumber_feature_30,copynumber_feature_31,copynumber_feature_32,copynumber_feature_33,copynumber_feature_34,copynumber_feature_35,copynumber_feature_36,copynumber_feature_37,copynumber_feature_38,copynumber_feature_39,...,copynumber_feature_61,copynumber_feature_62,copynumber_feature_63,copynumber_feature_64,copynumber_feature_65,copynumber_feature_66,copynumber_feature_67,copynumber_feature_68,copynumber_feature_69,copynumber_feature_70,copynumber_feature_71,copynumber_feature_72,copynumber_feature_73,copynumber_feature_74,copynumber_feature_75,copynumber_feature_76,copynumber_feature_77,copynumber_feature_78,copynumber_feature_79,copynumber_feature_80,copynumber_feature_81,copynumber_feature_82,copynumber_feature_83,copynumber_feature_84,copynumber_feature_85,copynumber_feature_86,copynumber_feature_87,copynumber_feature_88,copynumber_feature_89,copynumber_feature_90,copynumber_feature_91,copynumber_feature_92,copynumber_feature_93,copynumber_feature_94,copynumber_feature_95,copynumber_feature_96,copynumber_feature_97,copynumber_feature_98,copynumber_feature_99,Cell_line
0,-0.109201,-0.160136,-2.531141,0.021019,-0.378449,-0.291698,-0.690806,-1.068353,-1.038360,-0.173727,-1.076319,-0.045424,-0.906190,-0.185409,-0.418334,2.213546,-0.834868,-0.727916,-0.397759,-1.547042,-1.257846,1.498309,-0.486115,-1.565255,-0.531676,-2.419325,-0.763678,-0.928130,-0.776224,-0.738003,-0.822189,-1.254953,-1.043374,-0.601560,-0.913309,-0.377365,-0.220893,-0.843575,-0.312568,-1.294610,...,-1.561188,-1.569359,-0.152399,-0.094894,-0.726564,-0.200979,-1.470503,-0.316897,-0.103225,-0.298317,0.766120,0.394474,-0.497653,-0.786021,-1.384035,-0.377384,-0.630236,-0.832075,-0.214613,0.428468,1.346375,-1.213028,-1.425219,-0.671785,-0.885237,-0.637114,-0.231003,-0.835255,-0.396672,-0.872723,-1.486059,-1.016892,-1.526652,1.596862,-0.744194,-0.521416,-1.181389,-1.295292,-0.760382,M14
1,-0.325570,-0.785261,-3.176779,-0.795387,-0.259566,-0.321688,-0.215838,-1.145755,-0.361033,-0.009557,-1.479106,0.255681,-0.897880,-0.403064,-0.181555,1.436580,-0.159714,-0.738293,-0.387282,-0.970099,-0.827163,-0.753069,1.764941,-1.595275,-0.398178,-1.872913,0.681076,-0.638815,-0.430512,1.674277,-0.527286,-0.961032,-1.422899,0.256682,-1.575647,-1.474825,0.801652,-0.888368,-2.469102,-1.140743,...,-0.484389,-0.658274,-0.264124,-0.892126,-0.886500,-1.244537,-0.243777,-0.387718,-1.440380,-0.401305,-0.632281,1.306335,-0.875206,-1.194235,-1.776081,-0.563451,-0.068355,-0.471638,3.186594,-0.731198,-0.427989,-0.715988,-1.001317,-0.427283,-0.164892,-0.938388,-0.493095,-1.000592,0.447484,-0.123566,-4.175654,-0.891017,-2.647295,-0.150808,-0.408331,-1.310105,-3.441621,-0.687869,-0.007424,TE-12
2,-0.156567,-0.727474,-0.539222,-0.709865,-0.113751,-0.351196,-0.165670,-1.425358,-0.616899,-0.308831,-0.176213,0.068980,-1.436771,-0.227062,-0.431906,-0.069762,-0.155983,-0.938756,-0.848487,-1.268031,-0.756890,2.248810,-0.170544,-2.294326,-0.121383,-0.193952,-0.690122,-1.299559,-0.706316,0.457038,-0.209160,-0.802947,-0.767524,-0.285864,-0.703230,-1.172867,0.523440,-1.122247,-0.898149,-1.489581,...,-1.254114,-0.173183,-1.124363,-0.323065,-0.692566,0.435021,-0.422154,-0.258285,0.149669,-0.796491,0.332246,-0.036642,-0.893593,-1.170328,-1.147068,-0.593673,-0.036056,-0.589180,-0.154927,-0.034697,-0.135707,-0.242190,-0.453671,-0.618154,-0.843877,-1.799632,0.274148,-0.77

In [ ]:

from google.colab import files
encoded_train1.to_csv('copynumber_encoded.csv') 
files.download('copynumber_encoded.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>